# langchin_intro
    01_langchin_intro.ipynb


- LLM powered 어플리케이션 제작을 위한 프레임워크 

In [1]:
%pip install -q langchain langchain-openai

# -q : 실행 결과가 안 나오게

Note: you may need to restart the kernel to use updated packages.


In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano')
llm.invoke("Hello, world") # invoke 메쏘드 : 부르다.


# .env에 있는거 자동으로 갖고 오게 세팅이 바뀌었다네 

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 10, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-CArZtsSI1nej8ckdE8J0hmly0KrN9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--665c062c-9283-4d48-b57d-b3098718ad93-0', usage_metadata={'input_tokens': 10, 'output_tokens': 9, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
# 먼역 유리가, 외국어로 메시지가 들어오면, 한국어로 번역을 해주는 AI를 만들고 싶다면

# llm.invoke('한국어로 번역해줘: apple')

# 맥락이 이어지지 않는다. 내 생각에는 print 같은게 먼저 필요한데

msg = input('외국어를 넣으세요')

res = llm.invoke('한국어로 번역해줘: {msg}')

res.content 




'메세지를 입력해주세요.'

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [ 
    # SystemMessage(content='모든 영어를 한국어로 번역해라'),  # 채팅 세션의 전체적인 안내 사항, 시스템 프롬프트
    # HumanMessage(content='점심 먹어'),
    {'role': 'system', 'content' : 'Translate Korean to {lang}'}, # 위에꺼랑 같은 말
    # {'role': 'user', 'content' : '점심 먹어'}, # 위에꺼랑 같은 말
    {'role': 'human', 'content' : '점심 먹어'} # 위에꺼랑 같은 말
]


llm.invoke(messages)


AIMessage(content='Please eat lunch.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 81, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAqOMU9m29YcNRj63MiAmypCbGGZJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--89c3b468-b659-44fa-a4dc-b884fe24d22e-0', usage_metadata={'input_tokens': 81, 'output_tokens': 4, 'total_tokens': 85, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [16]:
for token in llm.stream(messages):
    print(token.content, end = "|")
    

|Please| have| lunch|.||

## Prompt Template 
- 고정된 문자열과 변수를 조합하여 프롬프트를 만드는 방법
- 'f스트링' 과 유사한

## chain 
- Langchain의 각 구성 요소를 묶어서 (chainning) 한 번에 실행(invoke) 할 수 있도록 하는 기능
- ' a | b | c' 형태로 나오는데 이건 파이썬 문법이 아니라 Langchain문법(LCEL- LanChain Expression Language)
- 체인의 모습이면 프롬프트가 먼저 

# 채팅을 할 경우에는 ChatPromptTemplate
# 단발성 명령 수행을 할 때는 from langchain_core.prompts import PromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

messages = [
    {'role': 'system', 'content' : 'Translate Korean to {lang}'},
    {'role': 'human', 'content' : '{text}'}

]

prompt_template = ChatPromptTemplate.from_messages(messages)

prompt = prompt_template.invoke({'lang':'일본어','text':'독도는 한국 땅'})

prompt.to_messages()

llm.invoke(prompt)


AIMessage(content='竹島は韓国の土地です。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 28, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAqfIxP9GlPsz1ZcqI24wmTYF9Wp0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7823de0f-f718-48b3-bd60-a677b95ead94-0', usage_metadata={'input_tokens': 28, 'output_tokens': 15, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# 체인으로 만들려고 하는 것 

prompt = prompt_template.invoke({'lang':'일본어','text':'독도는 한국 땅'})

llm.invoke(prompt)

AIMessage(content='竹島は韓国の土地です。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 28, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAqp905UJ7QeLcCmd3zzwbqNknqkG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c967ff4-fd48-4a5a-930b-42373ef5f9e2-0', usage_metadata={'input_tokens': 28, 'output_tokens': 15, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [29]:
from langchain_core.output_parsers import StrOutputParser 

output_parser = StrOutputParser()


chain = prompt_template | llm | output_parser

chain.invoke({'lang':'영어','text':'독도는 한국 땅이냐'})





'Dokdo is Korean territory.'

In [31]:
# 단발성 명령 수행을 할 때는 from langchain_core.prompts import PromptTemplate

from langchain_core.prompts import PromptTemplate

template = """
당신은 클라이언트의 요구사항을 분석해서 단계를 나눠주는 데이터 분석 전문가입니다.
사용자의 질문을 EDA를 할 수 있도록 단계를 나눠주세요.

상황: {question}

"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | output_parser

res = chain.invoke({'question':'소개팅에서 만난 상대가 2번째 데이트에서 별로일 때'})

print(res)

1. 데이터 수집 단계:
- 사용자의 소개팅 상대에 대한 정보를 수집합니다. (나이, 직업, 취미, 성격 등)
- 1차 데이트에서의 상대의 행동이나 대화 내용을 기록합니다.
- 2차 데이트에서의 상대의 행동이나 대화 내용을 기록합니다.

2. 데이터 정제 단계:
- 수집된 데이터 중에서 중요한 변수를 선별하여 정제합니다. (예: 관심사, 대화 내용, 감정 등)
- 비식별화를 위해 개인 정보를 제거하거나 익명화합니다.

3. 탐색적 데이터 분석(EDA) 단계:
- 1차 데이트와 2차 데이트에서의 상대의 특징을 비교하여 어떤 변화가 있는지 확인합니다.
- 상대의 성향이나 대화 내용을 통해 어떤 이유로 2차 데이트에서 별로였는지 파악합니다.
- 상대의 관심사나 성격을 분석하여 상대와의 호환성을 평가합니다.

4. 결과 해석 및 대안 제시 단계:
- EDA를 통해 파악한 결과를 바탕으로 상대와의 관계를 평가하고 다음 액션을 결정합니다.
- 다음 만남에서 어떤 점을 개선해야할지 아이디어를 제시하고, 상대와의 관계를 발전시킬 수 있는 방법을 고려합니다.


In [ ]:
# 강사님 코드
# # 단발성 명령 수행 PromptTemplate
from langchain_core.prompts import PromptTemplate

template = """
당신은 클라이언트 요구사항을 분석해서 단계를 나눠주는 데이터분석 전문가 입니다.
사용자의 질문을 EDA에 활용할 수 있도록 단계를 나눠주세요

질문: {question}
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | output_parser

res = chain.invoke({'question': '이번 2025년 전반기 매출 분석'})

print(res)

2025년 전반기 매출 분석을 위해 EDA(탐색적 데이터 분석) 단계를 다음과 같이 나눌 수 있습니다:

1. **목적 정의 및 데이터 수집**
   - 분석 목표 명확화: 2025년 전반기(1월~6월) 매출 동향 파악, 성장률 분석, 상품/지역별 매출 비교 등
   - 필요한 데이터 확보: 매출 데이터(거래일자, 상품, 고객, 지역, 결제 방법 등), 고객 정보, 프로모션 정보 등

2. **데이터 준비 및 정제**
   - 데이터 통합: 여러 소스의 데이터를 하나로 통합
   - 결측치 및 이상치 처리: 누락된 값, 급격한 이상치 검토 및 처리
   - 날짜 형식/시간대 정리: 거래 날짜를 적절히 정렬하고 필터링 가능하게 처리

3. **기초 통계 및 분포 파악**
   - 요약 통계: 매출, 거래 건수, 고객 수 등 기초 통계량 산출
   - 데이터 분포: 매출, 거래 건수의 분포 확인 (히스토그램, 박스플롯 등)
   - 월별/분기별 매출 추이: 시계열 성격 분석을 위한 기본 패턴 파악

4. **세부 분석**
   - 항목별 분석: 상품군, 지역, 고객 세그먼트별 매출 비중 및 트렌드 파악
   - 성장률 계산: 전년 동기 또는 전월 대비 성장률 분석
   - 고객 행동 분석: 재구매율, 고객군별 매출 기여도 등

5. **시각화 및 인사이트 도출**
   - 시계열 차트: 월별/전반기 매출 흐름 시각화
   - 파이 차트/바 차트: 상품군/지역별 매출 비중
   - 상관관계 분석: 주요 변수 간 관계 파악

6. **요약 및 보고**
   - 핵심 발견사항 정리
   - 추가 분석 필요 여부 검토
   - 최종 보고서 또는 대시보드 제작 제안

이 단계들을 따라 진행하면 2025년 전반기 매출에 대한 체계적이고 깊이 있는 탐색이 가능합니다.


In [44]:

# chain 은 Runnable 객체이기 때문에 invoke, stream, batch 메서드 사용가능
for token in chain.stream({'question': '이번 2025년 전반기 매출 분석'}):
    print(token, end='', flush=True)

# ???

2025년 전반기 매출 분석을 위해 데이터 분석 단계를 다음과 같이 나눌 수 있습니다:

1. **목표 정의 및 요구 사항 파악**
   - 분석 목적 확인: 2025년 전반기 매출 동향 파악, 성장 또는 감소 요인 분석 등
   - 주요 질문 도출: 월별 매출 추이, 지역별 차이, 상품군별 성과 등
   - 필요한 데이터 항목 확인

2. **데이터 수집**
   - 매출 데이터 확보: 월별 매출 기록, 상품/서비스 정보, 고객 데이터, 지역별 판매 데이터
   - 관련 자료: 프로모션, 할인행사, 이벤트 정보 등 추가 데이터 확보

3. **데이터 전처리 및 정제**
   - 결측치 처리, 이상치 제거
   - 데이터 형식 통일 (날짜, 금액 단위 등)
   - 파생 변수 생성 (예: 월, 분기, 연도 추출)
   - 데이터 통합 및 정렬

4. **기초 탐색적 데이터 분석 (EDA) 수행**
   - 전체 매출 추이 시각화 (월별, 누적 등)
   - 지역별, 상품별 매출 비중 분석
   - 시간에 따른 매출 패턴 파악
   - 주요 상승/하락 요인 탐색을 위한 변수 간 상관관계 분석

5. **심층 분석**
   - 성장/감소 원인 분석
   - 시즌별 매출 패턴 확인
   - 고객 세분화 및 행동 분석
   - 경쟁사 비교 또는 시장 동향 참고 (가능 시)

6. **인사이트 도출 및 결과 정리**
   - 핵심 인사이트 보고서 작성
   - 중요한 변수 파악 및 추천 사항 도출

7. **추후 분석 방향 및 전략 제언**
   - 마케팅 전략 제안
   - 매출 증대를 위한 방안 모색

이 단계들을 따라 진행하면 체계적이고 효과적인 2025년 전반기 매출 분석이 가능할 것입니다.

In [37]:
prompt = PromptTemplate.from_template('{topic}에 대해서 3문장으로 설명해줘')

llm = ChatOpenAI(model='gpt-4.1-nano')

chain = prompt | llm | StrOutputParser()

chain.batch([
    {'topic' :'Langchain'},
    {'topic' :'Langsmith'},
    {'topic' :'Langgraph'},
])


['LangChain은 자연어 처리(NLP) 애플리케이션을 개발할 때 사용되는 프레임워크로, LLM(대형 언어 모델)과 다양한 외부 도구를 연결하여 복잡한 작업을 수행할 수 있도록 도와줍니다. 이를 통해 정보 검색, 데이터 요약, 챗봇 등 다양한 응용 사례를 손쉽게 구현할 수 있으며, 모듈화된 구조를 제공하여 개발 효율성을 높여줍니다. 또한, 다양한 언어와 플랫폼을 지원하여 개발자가 유연하게 커스터마이징할 수 있도록 돕는 것이 특징입니다.',
 'Langsmith는 AI 모델과 사용자가 자연스럽게 소통할 수 있도록 돕는 대화형 AI 개발 플랫폼입니다. 사용자 친화적인 인터페이스와 다양한 도구를 제공하여 AI 모델의 커스터마이징과 배포를 쉽게 할 수 있습니다. 이를 통해 기업이나 개발자가 맞춤형 챗봇이나 대화형 애플리케이션을 효율적으로 만들 수 있습니다.',
 'Langgraph는 자연어 처리와 딥러닝 기술을 활용한 언어 이해 플랫폼으로, 텍스트 데이터를 그래프 구조로 모델링합니다. 이를 통해 문장 내 개체 간의 관계를 효과적으로 파악하고, 의미 분석 및 정보 추출이 가능합니다. Langgraph는 다양한 언어 처리 과제에 적용되어 더 정교한 언어 이해와 지능형 서비스 제공에 기여하고 있습니다.']